# Create methods that will batch data

In [1]:
import numpy as np

In [2]:
# Create a class that will do batching for the algorithm
class Audiobooks_Data_Reader():
    # Dataset is a mandatory argument, while batch_size is optional
    def __init__(self, dataset, batch_size=None):
    
        # The dataset that loads is one of 'train', 'val', or 'test'
        npz = np.load('Audiobooks_data_{0}.npz'.format(dataset))
        
        # Two variables that take the values of the inputs and targets
        self.inputs, self.targets = npz['inputs'].astype(float), npz['targets'].astype(int)
        
        # Counts the batch number, given the size you feed it later
        # If batch_size is None we are either validating or testing
        if batch_size is None:
            self.batch_size = self.inputs.shape[0]
        else:
            self.batch_size = batch_size
        self.curr_batch = 0
        self.batch_count = self.inputs.shape[0] // self.batch_size
        
    
    # A method which loads the next batch
    def __next__(self):
        if self.curr_batch >= self.batch_count:
            self.curr_batch = 0
            raise StopIteration()
            
            
        # You slice the dataset in batches and then the 'next' function 
        # loads them one after the other
        batch_slice = slice(self.curr_batch * self.batch_size, (self.curr_batch + 1) * self.batch_size)
        inputs_batch = self.inputs[batch_slice]
        targets_batch = self.targets[batch_slice]
        self.curr_batch += 1
        
        
        # One-hot encode the targets. In this example it's a bit unnecessary
        # since we have a 0/1 column as a target already but here's the code
        # anyway, as it'll be useful in future
        classes_num = 2
        targets_one_hot = np.zeros((targets_batch.shape[0], classes_num))
        targets_one_hot[range(targets_batch.shape[0]), targets_batch] = 1
        
        # The function will return the inputs batch and the one-hot encoded targets
        return inputs_batch, targets_one_hot
    
    
    # A method for iterating over the batches, as we will put them in a loop
    # This tells Python that the class we're defining is iterable
    # An iterator in Python is a class with the method __next__ and __iter__
    # that defines exactly how to iterate through its objects
    def __iter__(self):
        return self

# Explanation of above
- This class has methods init, next, and iter
- The **init** method loads the data from the .npz
- The **next** method loads the next batch from the .npz
- The **iter** tells Python that the class is iterable

This class is an iterator. An iterator is a class with methods: **next** and **iter**. Knowing this once a variable is an instance of the class, (e.g. train_data is an instance of this class) when included in the loop it will load the first batch. Then it will iterate over the dataset taking one batch after the other until the dataset is exhausted.



### init
- **self** is Python notation that defines the method as an instance method (see below)
This means init has only two real arguments:
- **dataset**
- **batch_size** is an optional argument, if we don't include it when calling the class, the class will assume it has to load all the data as a single batch

Example: Let x be an instance of Audiobooks_Data_Reader, where \
x = Audiobooks_Data_Reader('train', 5). \
Result: Load the data from Audiobooks_data_train.npz, take batches of 5 samples at a time.

Example: Let y be an instance of Audiobooks_Data_Reader, where \
y = Audiobooks_Data_Reader('val') \
Result: Load the data from Audiobooks_data_val.npz, take the whole dataset in a single batch.

### next
- This function slices the next batch out of the dataset and loads it for the next iteration
- This is also where we one-hot encode the targets, the targets change from 0 to \[1,0\] and from 1 to \[0,1\]. 
- The function will return the inputs batch and the one-hot encoded targets, this is also the only output we have associated with this class.

### Static vs Instance Methods
Imagine you have a class Person and an object that is instance of this class, e.g. Alice.

A static method .sayhi(), defined without self, would be called as: Person.sayhi(). This can be taken as a property of people in general and not just a particular individual, (i.e. a property of the class and not just of the instance)

One defined with self would be called as Alice.sayhi(), this is a property of the instance and not the class

# Create the ML algorithm

In [3]:
import tensorflow as tf

### Outline the model

In [4]:
input_size = 10
output_size = 2
hidden_size = 50

tf.compat.v1.reset_default_graph() 

# tf.reset_default_graph() clears the memory of all variables left
# from previous runs (reset the computational graph)

inputs = tf.compat.v1.placeholder(tf.float32, [None, input_size])
targets = tf.compat.v1.placeholder(tf.int32, [None, output_size])

weights_1 = tf.compat.v1.get_variable('weights_1', [input_size, hidden_size])
biases_1 = tf.compat.v1.get_variable('biases_1', [hidden_size])

# tf.get_variable('name', shape) is a function used to declare 
# variables. The default initializer is Xavier (Glorot)

outputs_1 = tf.nn.relu(tf.matmul(inputs, weights_1) + biases_1)

# tf.nn is a module that contains neural network support. Among 
# other things, it contains the most commonly used activation
# functions

weights_2 = tf.compat.v1.get_variable('weights_2', [hidden_size, hidden_size])
biases_2 = tf.compat.v1.get_variable('biases_2', [hidden_size])
outputs_2 = tf.nn.relu(tf.matmul(outputs_1, weights_2) + biases_2)

weights_3 = tf.compat.v1.get_variable('weights_3', [hidden_size, output_size])
biases_3 = tf.compat.v1.get_variable('biases_3', [output_size])

outputs = tf.matmul(outputs_2, weights_3) + biases_3

# It is common practice to incorporate the final activation in
# the loss

In [5]:
# Objective function
loss = tf.compat.v1.nn.softmax_cross_entropy_with_logits(logits=outputs, labels=targets)

# tf.nn.softmax_cross_entropy_with_logits(logits, labels) is a 
# function that applies a softmax activation and calculates a
# cross-entropy loss

mean_loss = tf.reduce_mean(loss)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [6]:
# Optimization
optimize = tf.compat.v1.train.AdamOptimizer(learning_rate=0.001).minimize(mean_loss)

In [7]:
# Prediction Accuracy
out_equals_target = tf.equal(tf.argmax(outputs,axis=1), tf.argmax(targets,axis=1))
accuracy = tf.reduce_mean(tf.cast(out_equals_target, tf.float32))

In [8]:
# Prepare for execution
sess = tf.compat.v1.InteractiveSession()

In [9]:
# Initializing variables
init = tf.compat.v1.global_variables_initializer()
sess.run(init)

In [10]:
# Batching
batch_size = 100

# Early stopping
max_epochs = 50

prev_val_loss = 9999999.

In [11]:
# Load data using class
train_data = Audiobooks_Data_Reader('train', batch_size)
val_data = Audiobooks_Data_Reader('val', batch_size)

In [12]:
# Make it learn

# Create a loop for the epochs. e is a variable that automatically
# starts from 0
for e in range(max_epochs):
    
    
    #TRAINING!!!!!!!!!
    # Keep track of sum of batch losses in the epoch
    epoch_loss = 0.
    
    # Iterate over the batches in this epoch using the CLASS
    for input_batch, target_batch in train_data:
        
        # Run the optimization step and get the mean loss for 
        # this batch. Feed it with the inputs and targets we just
        # got from the train set
        _, batch_loss = sess.run([optimize, mean_loss],
                                feed_dict={inputs:input_batch, targets:target_batch})
        
        # Increment the sum of the batch losses
        epoch_loss += batch_loss
        
    # Average batch loss    
    epoch_loss /= train_data.batch_count #this is the training loss
    
    
    # VALIDATION!!!!!!!!!!
    # Run without the optimization step (simply forward propagate)
    val_loss = 0.
    val_accuracy = 0.
    
    for input_batch, target_batch in val_data:
        val_loss, val_accuracy = sess.run([mean_loss, accuracy],
                                 feed_dict={inputs: input_batch, targets:target_batch})
        
    
    # Print stats for each epoch
    print('Epoch '+str(e+1)+
          '. Mean loss: '+'{0:.3f}'.format(epoch_loss)+
          '. Validation loss: '+'{0:.3f}'.format(val_loss)+
          '. Validation accuracy: '+'{0:.2f}'.format(val_accuracy * 100.)+'%')
    
    # Trigger early stopping if val_loss increases
    if val_loss > prev_val_loss:
        break
        
    # Store this epoch's val_loss to be used as prev_val_loss
    prev_val_loss = val_loss
    
        
# What does the inside of the 2nd for loop do?
# 1. Loads 100 inputs and targets (batch_size=100)
# 2. Optimizes the algorithm and calculates the batch loss
# 3. Records the loss for the iteration
# 4. Starts with the next 100 inputs and targets
# 5. Stops when the training set is exhausted

print('End of training')

Epoch 1. Mean loss: 0.615. Validation loss: 0.504. Validation accuracy: 76.00%
Epoch 2. Mean loss: 0.471. Validation loss: 0.425. Validation accuracy: 85.00%
Epoch 3. Mean loss: 0.422. Validation loss: 0.384. Validation accuracy: 85.00%
Epoch 4. Mean loss: 0.397. Validation loss: 0.359. Validation accuracy: 84.00%
Epoch 5. Mean loss: 0.381. Validation loss: 0.343. Validation accuracy: 84.00%
Epoch 6. Mean loss: 0.370. Validation loss: 0.330. Validation accuracy: 84.00%
Epoch 7. Mean loss: 0.362. Validation loss: 0.318. Validation accuracy: 84.00%
Epoch 8. Mean loss: 0.355. Validation loss: 0.310. Validation accuracy: 84.00%
Epoch 9. Mean loss: 0.350. Validation loss: 0.304. Validation accuracy: 85.00%
Epoch 10. Mean loss: 0.346. Validation loss: 0.299. Validation accuracy: 86.00%
Epoch 11. Mean loss: 0.342. Validation loss: 0.297. Validation accuracy: 86.00%
Epoch 12. Mean loss: 0.340. Validation loss: 0.294. Validation accuracy: 86.00%
Epoch 13. Mean loss: 0.337. Validation loss: 0.29

# Test the model

In [13]:
test_data = Audiobooks_Data_Reader('test')

for input_batch, target_batch in test_data:
    test_accuracy = sess.run([accuracy],
                    feed_dict = {inputs:input_batch, targets:target_batch})
    
test_accuracy_percent = test_accuracy[0]*100

print('Test accuracy: '+'{0:.2f}'.format(test_accuracy_percent)+'%')

Test accuracy: 81.25%
